In [1]:
import pandas as pd
import time
import random
import math
import mmh3

def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

In [3]:
def matching():
    global tp, fp, pairsNo, L1, q
    for index2 in range(nbS, nbS + offsetB): 
        if index2 > len(df2) - 1:
            return True

        rr = df2.iloc[index2, 0:5]
        idScholar = rr["id"]
        title = rr["title"]
        authors = rr["authors"]
        srec = title + " " + authors

        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                for id in ids:
                    if id in temp:
                        temp[id] += 1
                        if temp[id] / L1 >= t:
                            matchingPairs[id] = 1
                    else:
                        temp[id] = 1
        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == idScholar:
                        tp += 1
                        break
            else:
                fp += 1
    return False

def elimina_elementos_dentro_dictB(array_para_descarte, array_para_descarte_igual):
    array_para_descarte.clear()
    array_para_descarte_igual.clear()

In [4]:
df1 = pd.read_csv("../00-datasets/DBLP.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("../00-datasets/Scholar.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("../00-datasets/truth.csv", sep=",", encoding="utf-8", keep_default_na=False)
truthD = dict()
for i, r in truth.iterrows():
    idDBLP = r["idDBLP"]
    idScholar = r["idScholar"]
    if idDBLP in truthD:
        ids = truthD[idDBLP]
        ids.append(idScholar)
    else:
        truthD[idDBLP] = [idScholar]

t = 0.5
TP = 5347
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2

L= 116 L1= 50


In [4]:
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
lista_negra_indices = []
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0

tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array

tamanhoDosBlocos = {}

total_entidades = 0

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:5]
        idDBLP = rr["id"]
        title = rr["title"]
        authors = rr["authors"]

        srec = title + " " + authors
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                
                ids = d[key]
                ids_igual = d_igual[key]
                tamanho_atual = tamanhoDosBlocos[(key, l)]

                if len(ids) < tamanho_atual:
                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    elimina_elementos_dentro_dictB(ids, ids_igual)
                    lista_negra_indices.append((key, l))
                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
            else:
                d[key] = [idDBLP]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]

                tamanhoDosBlocos[(key, l)] = w
            
        tempoQueFoiInseridoNaEstrutura += 1

    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

blocking time (in mins) 0.10212192535400391
matching time (in mins) 2.6676968614260357
TP= 4689 Recall= 0.8769403403777819 Precision= 0.8931428571428571 pairsNo= 16359


In [5]:
lista_negra_indices

[('-2134634563', 115),
 ('-2121486483', 51),
 ('-2117574201', 49),
 ('-2136502599', 52),
 ('-2119024346', 56),
 ('-2133219679', 61),
 ('-2140351595', 20),
 ('-2144211646', 28),
 ('-2130995657', 100),
 ('-2141728181', 92),
 ('-2114357705', 83),
 ('-2136333345', 31),
 ('-2125379584', 96),
 ('-2120775340', 97),
 ('-2117355711', 91),
 ('-2139009139', 79),
 ('-2131355824', 67),
 ('-2131535071', 111),
 ('-2119150852', 69),
 ('-2131311958', 23),
 ('-2081592454', 44),
 ('-2114732221', 2),
 ('-2114644781', 81),
 ('-2066185946', 27),
 ('-2100239359', 0),
 ('-2136798172', 104),
 ('-2134621597', 78),
 ('-2130282845', 105),
 ('-2146651954', 3),
 ('-2121723674', 66),
 ('-2129781629', 57),
 ('-2093020375', 64),
 ('-2113456823', 11),
 ('-2142803252', 103),
 ('-2127222156', 43),
 ('-2070167981', 21),
 ('-2122007354', 15),
 ('-2138328710', 33),
 ('-2133641798', 1)]

## Teste com lista negra

In [ ]:
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
lista_negra_indices = []
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0

tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array

tamanhoDosBlocos = {}

total_entidades = 0

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:5]
        idDBLP = rr["id"]
        title = rr["title"]
        authors = rr["authors"]

        srec = title + " " + authors
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                
                ids = d[key]
                ids_igual = d_igual[key]
                tamanho_atual = tamanhoDosBlocos[(key, l)]

                if len(ids) < tamanho_atual:
                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    elimina_elementos_dentro_dictB(ids, ids_igual)
                    lista_negra_indices.append((key, l))

                    # ids.append(idDBLP)
                    # ids_igual.append(tempoQueFoiInseridoNaEstrutura)
            else:
                if (key, l) not in lista_negra_indices:                        
                    d[key] = [idDBLP]
                    d_igual[key] = [tempoQueFoiInseridoNaEstrutura]

                    tamanhoDosBlocos[(key, l)] = w
                else:
                    print("ignoring", key, l)
        tempoQueFoiInseridoNaEstrutura += 1

    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

blocking time (in mins) 0.09209954738616943
matching time (in mins) 2.433525482813517
TP= 4671 Recall= 0.8735739667103049 Precision= 0.8844915735656126 pairsNo= 16090


In [10]:
lista_negra_indices

[('-2134634563', 115),
 ('-2121486483', 51),
 ('-2117574201', 49),
 ('-2136502599', 52),
 ('-2119024346', 56),
 ('-2133219679', 61),
 ('-2140351595', 20),
 ('-2144211646', 28),
 ('-2130995657', 100),
 ('-2141728181', 92),
 ('-2114357705', 83),
 ('-2136333345', 31),
 ('-2125379584', 96),
 ('-2120775340', 97),
 ('-2117355711', 91),
 ('-2139009139', 79),
 ('-2131355824', 67),
 ('-2131535071', 111),
 ('-2119150852', 69),
 ('-2131311958', 23),
 ('-2081592454', 44),
 ('-2114732221', 2),
 ('-2114644781', 81),
 ('-2066185946', 27),
 ('-2100239359', 0),
 ('-2136798172', 104),
 ('-2134621597', 78),
 ('-2130282845', 105),
 ('-2146651954', 3),
 ('-2121723674', 66),
 ('-2129781629', 57),
 ('-2093020375', 64),
 ('-2113456823', 11),
 ('-2142803252', 103),
 ('-2127222156', 43),
 ('-2070167981', 21),
 ('-2122007354', 15),
 ('-2138328710', 33),
 ('-2133641798', 1)]

In [14]:
teste = [e for e, _ in enumerate(lista_negra_indices)]
teste2 = set(teste)
len(teste2), len(teste)

(39, 39)

In [15]:
if ('-2134634563', 115) in lista_negra_indices:
    print("VAMOOO")

VAMOOO


# Apagar estrutura de tempos em tempos

In [5]:
mock_data = {
    # ID 1: Artigos de SIGMOD (6 entradas)
    '-2053413593': [
        'journals/sigmod/EisenbergM02',
        'conf/sigmod/HaasNSS95',
        'conf/sigmod/HoelS95',
        'journals/sigmod/SlivinskasJS02',
        'conf/sigmod/WienerN95',
        'conf/sigmod/KaushikBNK02'
    ],
    
    # ID 2: VLDB e SIGMOD (4 entradas)
    '-1844936030': [
        'conf/vldb/HungYK96',
        'journals/vldb/ClaussenKKW00',
        'conf/sigmod/GarciaP99',
        'journals/vldb/LiT22'
    ],
    
    # ID 3: Workshops (3 entradas)
    '-1606265440': [
        'workshops/dbsec/ZhangL19',
        'workshops/clouddb/ChenR21',
        'conf/icde/workshops/WangX23'
    ],
    
    # ID 4: Artigos recentes (2 entradas)
    '-1834122268': [
        'conf/cidr/WangC23',
        'journals/pvldb/LeeM22'
    ],
    
    # ID 5: Keynotes (1 entrada)
    '-1881148935': [
        'conf/vldb/keynotes/Uhl22'
    ],
    
    # ID 6: Artigos clássicos (3 entradas)
    '-2009778662': [
        'journals/vldb/OzsuPSILM95',
        'conf/pods/AbiteboulH95',
        'journals/tods/Gray78'
    ],
    
    # ID 7: Machine Learning em BD (2 entradas)
    '-1999999999': [
        'conf/sigmod/KumarML21',
        'journals/pvldb/RatanM22'
    ],
    
    # ID 8: Bancos de Dados Não-Relacionais (3 entradas)
    '-1111111111': [
        'conf/nosql/BrownN20',
        'journals/nosql/WhiteD21',
        'conf/icde/NoSQL/ClarkP22'
    ]
}

# DictBzinho com 2 posições PREENCHIDAS (sem dicionários vazios)
dictBzinho = [
    # Primeiro dicionário em dictBzinho (Tema: Otimização de Consultas)
    {
        '-555555555': [
            'conf/sigmod/StonebrakerR00',
            'journals/vldb/Chaudhuri98',
            'conf/icde/Graefe93'
        ],
        '-666666666': [
            'workshops/query_opt/ZhouK21',
            'conf/pods/GarciaMolina02'
        ]
    },
    
    # Segundo dicionário em dictBzinho (Tema: Bancos de Dados Distribuídos)
    {
        '-777777777': [
            'journals/distributed/Bernstein87',
            'conf/vldb/ÖzsuT19'
        ],
        '-888888888': [
            'conf/icdcn/DistributedDB/LiH23',
            'journals/pvldb/KemmeA22',
            'conf/sigmod/Distributed/ShahP21'
        ]
    }
]

# Estrutura final combinada
final_structure = [mock_data] + dictBzinho

mock_data1 = {
    # ID 1: Artigos de SIGMOD (6 números ordenados)
    '-2053413593': [5, 7, 12, 23, 34, 41],
    
    # ID 2: VLDB e SIGMOD (4 números ordenados)
    '-1844936030': [3, 18, 29, 45],
    
    # ID 3: Workshops (3 números ordenados)
    '-1606265440': [9, 14, 27],
    
    # ID 4: Artigos recentes (2 números ordenados)
    '-1834122268': [21, 33],
    
    # ID 5: Keynotes (1 número)
    '-1881148935': [2],
    
    # ID 6: Artigos clássicos (3 números ordenados)
    '-2009778662': [8, 15, 37],
    
    # ID 7: Machine Learning em BD (2 números ordenados)
    '-1999999999': [19, 46],
    
    # ID 8: Bancos de Dados Não-Relacionais (3 números ordenados)
    '-1111111111': [11, 30, 48]
}

# DictBzinho com 2 posições PREENCHIDAS com números ordenados
dictBzinho1 = [
    # Primeiro dicionário (Tema: Otimização de Consultas)
    {
        '-555555555': [4, 22, 39],
        '-666666666': [16, 50]
    },
    
    # Segundo dicionário (Tema: Bancos de Dados Distribuídos)
    {
        '-777777777': [1, 25],
        '-888888888': [10, 31, 42]
    }
]

# Estrutura final combinada
final_structure1 = [mock_data1] + dictBzinho1

In [8]:
final_structure

[{'-2053413593': ['journals/sigmod/EisenbergM02',
   'conf/sigmod/HaasNSS95',
   'conf/sigmod/HoelS95',
   'journals/sigmod/SlivinskasJS02',
   'conf/sigmod/WienerN95',
   'conf/sigmod/KaushikBNK02'],
  '-1844936030': ['conf/vldb/HungYK96',
   'journals/vldb/ClaussenKKW00',
   'conf/sigmod/GarciaP99',
   'journals/vldb/LiT22'],
  '-1606265440': ['workshops/dbsec/ZhangL19',
   'workshops/clouddb/ChenR21',
   'conf/icde/workshops/WangX23'],
  '-1834122268': ['conf/cidr/WangC23', 'journals/pvldb/LeeM22'],
  '-1881148935': ['conf/vldb/keynotes/Uhl22'],
  '-2009778662': ['journals/vldb/OzsuPSILM95',
   'conf/pods/AbiteboulH95',
   'journals/tods/Gray78'],
  '-1999999999': ['conf/sigmod/KumarML21', 'journals/pvldb/RatanM22'],
  '-1111111111': ['conf/nosql/BrownN20',
   'journals/nosql/WhiteD21',
   'conf/icde/NoSQL/ClarkP22']},
 {'-555555555': ['conf/sigmod/StonebrakerR00',
   'journals/vldb/Chaudhuri98',
   'conf/icde/Graefe93'],
  '-666666666': ['workshops/query_opt/ZhouK21', 'conf/pods/Ga

In [7]:
final_structure1

[{'-2053413593': [5, 7, 12, 23, 34, 41],
  '-1844936030': [3, 18, 29, 45],
  '-1606265440': [9, 14, 27],
  '-1834122268': [21, 33],
  '-1881148935': [2],
  '-2009778662': [8, 15, 37],
  '-1999999999': [19, 46],
  '-1111111111': [11, 30, 48]},
 {'-555555555': [4, 22, 39], '-666666666': [16, 50]},
 {'-777777777': [25], '-888888888': [10, 31, 42]}]

In [6]:
def elimina_entidades_antigas_global(final_structure, final_structure1, tempoQueFoiInseridoNaEstrutura):
    for indice in range(len(final_structure)):
        # print(dictB[indice])
        bloco_unico = final_structure[indice]
        for key, array in bloco_unico.items():
            # print(f"key {key} e o array é {array}")
            # print(f"final_structure1 {final_structure1[indice][key]}")
            tempos = final_structure1[indice][key]            
            idx = 0
            while idx < len(tempos) and tempos[idx] < tempoQueFoiInseridoNaEstrutura:
                idx += 1
            if idx > 0:
                print(f"tem que deletar até o índice {idx} (valores menores que {tempoQueFoiInseridoNaEstrutura}), no", final_structure[indice][key])
                del final_structure[indice][key][:idx]
                del final_structure1[indice][key][:idx]

elimina_entidades_antigas_global(final_structure, final_structure1, 2)

tem que deletar até o índice 1 (valores menores que 2), no ['journals/distributed/Bernstein87', 'conf/vldb/ÖzsuT19']


In [ ]:
def elimina_entidades_antigas_global(final_structure, final_structure1, tempoQueFoiInseridoNaEstrutura, indice_atualllll):
    # print("entrou aqui")
    for indice in range(len(final_structure1)):
        # print(dictB[indice])
        bloco_unico = final_structure1[indice]
        for key, array in bloco_unico.items():
            # print(f"dictB_igual {dictB_igual[indice][key]}")
            tempos = final_structure1[indice][key]  

            idx = 0
            # print(tempoQueFoiInseridoNaEstrutura)
            while idx < len(tempos) and tempos[idx] < tempoQueFoiInseridoNaEstrutura:
                idx += 1
                # print("passou aqui")
            if idx > 0:
                # print("\n")
                # print(f"indice_atualllll {indice_atualllll}")
                # print(tempoQueFoiInseridoNaEstrutura)
                # print("Antes de deletar", final_structure1[indice][key])
                # print(f"tem que deletar até o índice {idx} (valores menores que {tempoQueFoiInseridoNaEstrutura}), no", final_structure[indice][key])
                del final_structure[indice][key][:idx]
                del final_structure1[indice][key][:idx]
                # print("depois de deletar", final_structure1[indice][key])


In [10]:
elimina_entidades_antigas_global(final_structure, final_structure1, 2, 1)

In [61]:
def elimina_entidades_antigas_global(dictB, dictB_igual, tempoQueFoiInseridoNaEstrutura, indice_atualllll):
    # print("entrou aqui")
    for indice in range(len(dictB_igual)):
        # print(dictB[indice])
        bloco_unico = dictB_igual[indice]
        for key, array in bloco_unico.items():
            # print(f"dictB_igual {dictB_igual[indice][key]}")
            tempos = dictB_igual[indice][key]  
                 
            idx = 0
            # print(tempoQueFoiInseridoNaEstrutura)
            while idx < len(tempos) and tempos[idx] < tempoQueFoiInseridoNaEstrutura:
                idx += 1
                # print("passou aqui")
            if idx > 0:
                # print("\n")
                # print(f"indice_atualllll {indice_atualllll}")
                # print(tempoQueFoiInseridoNaEstrutura)
                # print("Antes de deletar", dictB_igual[indice][key])
                # print(f"tem que deletar até o índice {idx} (valores menores que {tempoQueFoiInseridoNaEstrutura}), no", dictB[indice][key])
                del dictB[indice][key][:idx]
                del dictB_igual[indice][key][:idx]
                # print("depois de deletar", dictB_igual[indice][key])


In [ ]:
def elimina_entidades_antigas_global_ttttttt(dictB, dictB_igual, total_entidades, ciclo_atual):
    # Calcula quantas entidades devem ser removidas (20% das adicionadas no ciclo)
    entidades_para_remover = max(100, total_entidades // 5)  # Pelo menos 100 ou 20% do total
    
    for indice in range(len(dictB_igual)):
        bloco_unico = dictB_igual[indice]
        for key, array in bloco_unico.items():
            tempos = dictB_igual[indice][key]
            # Ordena os tempos se necessário (assumindo que já estão ordenados por inserção)
            # Remove as entidades mais antigas (primeiras da lista)
            if len(tempos) > entidades_para_remover:
                del dictB[indice][key][:entidades_para_remover]
                del dictB_igual[indice][key][:entidades_para_remover]
            else:
                # Se o bloco tem menos entidades que o necessário para remover, remove tudo
                dictB[indice].pop(key, None)
                dictB_igual[indice].pop(key, None)

In [ ]:
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]

tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0

tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array

tamanhoDosBlocos = {}

total_entidades = 0
indice_atualllll = 0
while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:5]
        idDBLP = rr["id"]
        title = rr["title"]
        authors = rr["authors"]

        srec = title + " " + authors
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                
                ids = d[key]
                ids_igual = d_igual[key]
                tamanho_atual = tamanhoDosBlocos[(key, l)]

                if len(ids) < tamanho_atual:
                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    elimina_elementos_dentro_dictB(ids, ids_igual)
                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
            else:
                d[key] = [idDBLP]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]

                tamanhoDosBlocos[(key, l)] = w
        
        tempoQueFoiInseridoNaEstrutura += 1
        # A cada 1000 inserções, remove entidades antigas para economizar memória
        if tempoQueFoiInseridoNaEstrutura % 500 == 0:
            indice_atualllll += 1
            # Define um limite de "idade" para entidades antigas (por exemplo, 900 atrás)
            limite_idade = 100 * indice_atualllll  # Aumenta o limite de idade a cada iteração
            print(limite_idade)
            # ver_passado = tempoQueFoiInseridoNaEstrutura - limite_idade
            # tempo_limite = ver_passado if ver_passado > 0 else 0
            elimina_entidades_antigas_global_ttttttt(dictB, dictB_igual, limite_idade, indice_atualllll)
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

100
200
300
400
500
blocking time (in mins) 0.10256336530049642
matching time (in mins) 2.622007199128469
TP= 4170 Recall= 0.7798765662988592 Precision= 0.8252523253512765 pairsNo= 25817


In [63]:
tempoQueFoiInseridoNaEstrutura

2615

In [ ]:
#retira até 1
final_structure1

[{'-2053413593': [5, 7, 12, 23, 34, 41],
  '-1844936030': [3, 18, 29, 45],
  '-1606265440': [9, 14, 27],
  '-1834122268': [21, 33],
  '-1881148935': [2],
  '-2009778662': [8, 15, 37],
  '-1999999999': [19, 46],
  '-1111111111': [11, 30, 48]},
 {'-555555555': [4, 22, 39], '-666666666': [16, 50]},
 {'-777777777': [25], '-888888888': [10, 31, 42]}]

In [ ]:
#retira até 0
final_structure1

[{'-2053413593': [5, 7, 12, 23, 34, 41],
  '-1844936030': [3, 18, 29, 45],
  '-1606265440': [9, 14, 27],
  '-1834122268': [21, 33],
  '-1881148935': [2],
  '-2009778662': [8, 15, 37],
  '-1999999999': [19, 46],
  '-1111111111': [11, 30, 48]},
 {'-555555555': [4, 22, 39], '-666666666': [16, 50]},
 {'-777777777': [1, 25], '-888888888': [10, 31, 42]}]

In [ ]:
#Retirar até 10
final_structure1

[{'-2053413593': [12, 23, 34, 41],
  '-1844936030': [18, 29, 45],
  '-1606265440': [14, 27],
  '-1834122268': [21, 33],
  '-1881148935': [],
  '-2009778662': [15, 37],
  '-1999999999': [19, 46],
  '-1111111111': [11, 30, 48]},
 {'-555555555': [22, 39], '-666666666': [16, 50]},
 {'-777777777': [25], '-888888888': [10, 31, 42]}]

In [ ]:
# retirar até 35
final_structure1

[{'-2053413593': [41],
  '-1844936030': [45],
  '-1606265440': [],
  '-1834122268': [],
  '-1881148935': [],
  '-2009778662': [37],
  '-1999999999': [46],
  '-1111111111': [48]},
 {'-555555555': [39], '-666666666': [50]},
 {'-777777777': [], '-888888888': [42]}]